In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgbm
import gc

from sklearn.linear_model import Lasso, ElasticNet, RANSACRegressor, Ridge
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import LinearSVR,SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from itertools import product

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
%matplotlib inline

import seaborn as sns

### seaborn에서 한글 나오게하기
sns.set(font="New Gulim")

import zipfile
import stacking
from sklearn import preprocessing


c:\users\beoms\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load Data

In [2]:
train = pd.read_csv('train_2016.csv')
prop = pd.read_csv('properties_2016.csv')
sample = pd.read_csv('sample_submission.csv')

c:\users\beoms\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(prop['taxdelinquencyyear'])

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          NaN
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
21          NaN
22         13.0
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
           ... 
2985187     NaN
2985188     NaN
2985189     NaN
2985190     NaN
2985191     NaN
2985192     NaN
2985193     NaN
2985194     NaN
2985195     NaN
2985196     NaN
2985197     NaN
2985198     NaN
2985199     NaN
2985200     NaN
2985201     NaN
2985202     NaN
2985203     NaN
2985204     NaN
2985205     NaN
2985206     NaN
2985207     NaN
2985208     NaN
2985209     NaN
2985210     NaN
2985211     NaN
2985212     NaN
2985213     NaN
2985214     NaN
2985215     NaN
2985216     NaN
Name: taxdelinquencyyear

In [3]:
from imp import reload
reload(stacking)

<module 'stacking' from 'C:\\Users\\beoms\\workspace\\kaggle\\stacking.py'>

In [4]:
df_train = train.merge(prop, how='left', on='parcelid')

In [5]:
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

In [6]:
df_train.loc[df_train['hashottuborspa'].isnull(),'hashottuborspa'] = False
df_test.loc[df_test['hashottuborspa'].isnull(),'hashottuborspa'] = False

df_train.loc[df_train['taxdelinquencyflag'].isnull(),'taxdelinquencyflag'] = 'N'
df_test.loc[df_test['taxdelinquencyflag'].isnull(),'taxdelinquencyflag'] = 'N'

df_train.loc[df_train['fireplaceflag'].isnull(),'fireplaceflag'] = False
df_test.loc[df_test['fireplaceflag'].isnull(),'fireplaceflag'] = False

df_train['transactiondate_month'] = pd.to_datetime(df_train['transactiondate']).dt.month
df_train['month_bedrooms'] = df_train['bedroomcnt'] * df_train['transactiondate_month']
del df_train['transactiondate']

In [7]:
def feature_fillna(df_data):
    # should be filled
    df_data.loc[df_data['decktypeid'].isnull(),'decktypeid'] = 0.
    df_data.loc[df_data['fireplaceflag'].isnull(),'fireplaceflag'] = False
    df_data.loc[df_data['fireplacecnt'].isnull(),'fireplacecnt'] = 0.
    df_data.loc[df_data['hashottuborspa'].isnull(),'hashottuborspa'] = False
    df_data.loc[df_data['poolcnt'].isnull(),'poolcnt'] = 0.
    df_data.loc[df_data['poolsizesum'].isnull(),'poolsizesum'] = 0.    
    df_data.loc[df_data['pooltypeid10'].isnull(),'pooltypeid10'] = 0.
    df_data.loc[df_data['pooltypeid2'].isnull(),'pooltypeid2'] = 0.
    df_data.loc[df_data['pooltypeid7'].isnull(),'pooltypeid7'] = 0.
    df_data.loc[df_data['storytypeid'].isnull(),'pooltypeid7'] = 0.
    df_data.loc[df_data['taxdelinquencyflag'].isnull(),'taxdelinquencyflag'] = 'N'
    df_data.loc[df_data['basementsqft'].isnull(),'basementsqft'] = 0.
    df_data.loc[df_data['garagecarcnt'].isnull(),'garagecarcnt'] = 0.
    df_data.loc[df_data['garagetotalsqft'].isnull(),'garagetotalsqft'] = 0.
    df_data.loc[df_data['yardbuildingsqft17'].isnull(),'yardbuildingsqft17'] = 0.
    df_data.loc[df_data['yardbuildingsqft26'].isnull(),'yardbuildingsqft26'] = 0.    
    df_data.loc[df_data['structuretaxvaluedollarcnt'].isnull(),'structuretaxvaluedollarcnt'] = 0.

    # may be filled
    df_data.loc[df_data['airconditioningtypeid'].isnull(),'airconditioningtypeid'] = 0.
    df_data.loc[df_data['numberofstories'].isnull(),'numberofstories'] = 0.
    df_data.loc[df_data['storytypeid'].isnull(),'storytypeid'] = 0.
    df_data.loc[df_data['threequarterbathnbr'].isnull(),'threequarterbathnbr'] = 0.
    df_data.loc[df_data['taxdelinquencyyear'].isnull(),'taxdelinquencyyear'] = 0.
    
    # may not but,,
    df_data.loc[df_data['unitcnt'].isnull(),'unitcnt'] = -1.
    df_data.loc[df_data['heatingorsystemtypeid'].isnull(),'heatingorsystemtypeid'] = -1.

In [8]:
feature_fillna(df_train)
feature_fillna(df_test)

In [9]:
# Label Encoding을 할 때는 항상 train과 test를 합친 전체 value를 fit하고 transform 해줘야 합니다.
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        print(col)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[col].values) + list(df_test[col].values))
        df_train[col] = lbl.transform(list(df_train[col].values))
        df_test[col] = lbl.transform(list(df_test[col].values))

hashottuborspa
propertycountylandusecode
propertyzoningdesc
fireplaceflag
taxdelinquencyflag


In [10]:
#drop outlier
df_train=df_train[ df_train.logerror > -0.4 ]
df_train=df_train[ df_train.logerror < 0.419 ]

In [11]:
def drop_features(df_data):
    # drop!
    df_data.drop('finishedsquarefeet13', axis=1, inplace=True)
    df_data.drop('architecturalstyletypeid', axis=1, inplace=True)
    df_data.drop('typeconstructiontypeid', axis=1, inplace=True)
    df_data.drop('finishedsquarefeet6', axis=1, inplace=True)    
    
    # maybe not but.
    df_data.drop('buildingclasstypeid', axis=1, inplace=True)
    df_data.drop('finishedsquarefeet15', axis=1, inplace=True)
    df_data.drop('finishedfloor1squarefeet', axis=1, inplace=True)
    df_data.drop('finishedsquarefeet50', axis=1, inplace=True)
    df_data.drop('regionidneighborhood', axis=1, inplace=True)  

In [12]:
drop_features(df_train)
drop_features(df_test)

In [13]:
x_train = df_train.drop(['parcelid', 'logerror'], axis=1)
train_columns = x_train.columns

y_train = df_train['logerror']
y_mean = np.mean(y_train)

df_test['transactiondate_month'] = 10
df_test['month_bedrooms'] = df_test['transactiondate_month']*df_test['bedroomcnt']
x_test = df_test[train_columns]
print(x_train.shape, y_train.shape, x_test.shape)

del df_train,df_test
gc.collect()

(88528, 50) (88528,) (2985217, 50)


388

# Stacking Model

In [51]:
et_parmas = {
    'criterion':'mse', 'max_leaf_nodes':30, 'n_estimators':1000, 'min_impurity_split':0.0000001,
    'max_features':0.6, 'max_depth':10, 'min_samples_leaf':20, 'min_samples_split':2,
    'min_weight_fraction_leaf':0.0, 'bootstrap':True,
    'random_state':1, 'verbose':False
    
}

In [52]:




et_model = stacking.SklearnWrapper(clf = ExtraTreesRegressor,params=rf_params)


In [53]:


et_train,et_test = stacking.get_oof(et_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)


ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=6,
          max_features=0.25, max_leaf_nodes=30, min_impurity_split=1e-07,
          min_samples_leaf=20, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
          oob_score=False, random_state=0, verbose=False, warm_start=False)
Fold 1 /  CV-Score: 0.053161


MemoryError: 

In [ ]:
x_train_second_layer = np.concatenate((lgbm1_train, lgbm2_train, lgbm3_train,
                         lgbm4_train, xgb1_train, xgb2_train,
                         gbm1_train,ridge_train, lasso_train, rf_train,et_train), axis=1)

In [ ]:
x_test_second_layer = np.concatenate((lgbm1_test, lgbm2_test, lgbm3_test,
                        lgbm4_test,xgb1_test,xgb2_test,
                       gbm1_test,ridge_test,lasso_test,rf_test,et_test), axis=1)

### Param

In [13]:
lgbm_params1 = {
    'boosting':'gbdt', 'num_leaves':10, 'learning_rate':0.01, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':4, 'feature_fraction':0.5, 'min_data_in_leaf':4, 'poission_max_delta_step':0.7,
    'bagging_fraction':0.8, 'min_gain_to_split':0, 'scale_pos_weight':1.0,
    'lambda_l2':0.1, 'lambda_l1':0.1, 'huber_delta':1.0, 'bagging_freq':1,
    'objective':'regression_l1', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False,
    'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1,
    'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'mae','threads':6
}

lgbm_params2 = {
    'boosting':'gbdt', 'num_leaves':24,'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':6, 'feature_fraction':0.5, 'min_data_in_leaf':50, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8,
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'huber_delta':0.05,
    'bagging_freq':1, 'objective':'huber', 'seed':1, 'categorical_feature':0 ,'xgboost_dart_mode':False, 'drop_rate':0.1,
    'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50,
    'bin_construct_sample_cnt':1000000, 'two_round':False, 'uniform_drop':False,'metric': 'mae','threads':6
}

lgbm_params3 = {
    'boosting':'gbdt', 'num_leaves':28, 'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1, 'max_depth':7,
    'feature_fraction':0.6, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8,
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'fair_c':0.01, 'bagging_freq':1,
    'objective':'fair', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False, 'drop_rate':0.1, 'skip_drop':0.5,
    'max_drop':50, 'top_rate':0.1, 'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'mae','threads':6
}

lgbm_params4 = {
    'boosting':'gbdt', 'num_leaves':16, 'learning_rate':0.003, 'min_sum_hessian_in_leaf':0.1, 'max_depth':7,
    'feature_fraction':0.5, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7, 'bagging_fraction':0.8, 
    'min_gain_to_split':0, 'scale_pos_weight':1.0, 'lambda_l2':0.1, 'lambda_l1':0.1, 'bagging_freq':1, 'objective':'regression',
    'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False, 'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 
    'top_rate':0.1,'other_rate':0.1, 'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000, 
    'two_round':False,'uniform_drop':False,'metric': 'mae','threads':6
}

xgb_params1 = {
    'booster':'gbtree', 'objective':'reg:linear', 'max_leaves':0, 'eta':0.02, 'gamma':1,
    'max_depth':4, 'colsample_bylevel':1.0, 'min_child_weight':4.0, 'max_delta_step':0.0, 'subsample':0.8, 
    'colsample_bytree':0.5,'scale_pos_weight':1.0, 'alpha':1.0, 'lambda':5.0, 'seed':1
}

xgb_params2 = {
    'booster':'gblinear', 'objective':'reg:linear', 'max_leaves':0, 'eta':0.1,'gamma':1,
    'max_depth':4, 'colsample_bylevel':1.0, 'min_child_weight':4.0, 'max_delta_step':0.0, 'subsample':0.8, 
    'colsample_bytree':0.5,'scale_pos_weight':1.0, 'alpha':10.0, 'lambda':1.0, 'seed':1
}

sgd_param = {
    'loss':'huber','penalty':'l2','alpha':1,'l1_ratio':0.15,'eta0':0.001,
    'fit_intercept':True,'shuffle':True,'random_state':1,
}

gbm_param = {
    'n_estimators' :100, 'learning_rate':0.1, 'min_samples_split' :0.00001,
    'subsample':1.0, 'max_depth':5, 'max_features':0.4,
    'min_samples_leaf' :4.0, 'random_state' :1
}

lasso_params={
    'alpha':0.003,
    'normalize':True,
    'max_iter':200,'fit_intercept':True,'tol':0.007,
    'warm_start':True
}

ridge_params={
    'alpha':0.2,
    'normalize':True,
    'max_iter':200,'fit_intercept':False,'solver':'auto'
}

rf_params = {
    'criterion':'mse', 'max_leaf_nodes':30, 'n_estimators':1000, 'min_impurity_split':0.0000001,
    'max_features':0.25, 'max_depth':6, 'min_samples_leaf':20, 'min_samples_split':2,
    'min_weight_fraction_leaf':0.0, 'bootstrap':True,
    'random_state':1, 'verbose':False
    
}

et_parmas = {
    'criterion':'mse', 'max_leaf_nodes':30, 'n_estimators':1000, 'min_impurity_split':0.0000001,
    'max_features':0.6, 'max_depth':10, 'min_samples_leaf':20, 'min_samples_split':2,
    'min_weight_fraction_leaf':0.0, 'bootstrap':True,
    'random_state':1, 'verbose':False
    
}

In [ ]:
keras_model = stacking.kerasWrapper(clf = ExtraTreesRegressor,params=rf_params)

### Model

In [15]:
lgbm_model1 = stacking.LgbmWrapper(params=lgbm_params1, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model2 = stacking.LgbmWrapper(params=lgbm_params2, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model3 = stacking.LgbmWrapper(params=lgbm_params3, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

lgbm_model4 = stacking.LgbmWrapper(params=lgbm_params4, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

xgb_model1 = stacking.XgbWrapper(params=xgb_params1, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

xgb_model2 = stacking.XgbWrapper(params=xgb_params2, num_rounds = 1500, ealry_stopping=100,
                                   verbose_eval=False, base_score=True, maximize=False,
                                   y_value_log=False)

gbm_model = stacking.SklearnWrapper(clf = GradientBoostingRegressor,params=gbm_param)

ridge_model = stacking.SklearnWrapper(clf = Ridge,params=ridge_params)

lasso_model = stacking.SklearnWrapper(clf = Lasso,params=lasso_params)

rf_model = stacking.SklearnWrapper(clf = RandomForestRegressor,params=rf_params)

et_model = stacking.SklearnWrapper(clf = ExtraTreesRegressor,params=rf_params)

### Run!

In [32]:
lgbm1_train,lgbm1_test = stacking.get_oof(lgbm_model1,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
lgbm2_train,lgbm2_test = stacking.get_oof(lgbm_model2,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
lgbm3_train,lgbm3_test = stacking.get_oof(lgbm_model3,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
lgbm4_train,lgbm4_test = stacking.get_oof(lgbm_model4,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
gc.collect()
xgb1_train,xgb1_test = stacking.get_oof(xgb_model1,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
xgb2_train,xgb2_test = stacking.get_oof(xgb_model2,x_train,y_train,x_test, mean_absolute_error,NFOLDS=5)
gc.collect()
gbm1_train,gbm1_test = stacking.get_oof(gbm_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)
ridge_train,ridge_test = stacking.get_oof(ridge_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)
lasso_train,lasso_test = stacking.get_oof(lasso_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)
rf_train,rf_test = stacking.get_oof(rf_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)
et_train,et_test = stacking.get_oof(et_model,x_train.fillna(-1),y_train,x_test.fillna(-1), mean_absolute_error,NFOLDS=5)
gc.collect()

{'min_sum_hessian_in_leaf': 0.1, 'categorical_feature': 0, 'bagging_freq': 1, 'boosting': 'gbdt', 'max_drop': 50, 'seed': 0, 'drop_rate': 0.1, 'lambda_l2': 0.1, 'feature_fraction': 0.5, 'objective': 'regression_l1', 'uniform_drop': False, 'skip_drop': 0.5, 'min_data_in_leaf': 4, 'num_leaves': 10, 'two_round': False, 'lambda_l1': 0.1, 'bin_construct_sample_cnt': 1000000, 'xgboost_dart_mode': False, 'max_bin': 255, 'min_gain_to_split': 0, 'scale_pos_weight': 1.0, 'other_rate': 0.1, 'max_depth': 4, 'bagging_fraction': 0.8, 'poission_max_delta_step': 0.7, 'metric': 'mae', 'learning_rate': 0.01, 'huber_delta': 1.0, 'min_data_in_bin': 50, 'top_rate': 0.1}
Fold 1 /  CV-Score: 0.053236
Fold 2 /  CV-Score: 0.052129
Fold 3 /  CV-Score: 0.053018
Fold 4 /  CV-Score: 0.053067
Fold 5 /  CV-Score: 0.052961
Average CV-Score:  0.052882322209


### Second Layer

In [ ]:
x_train_second_layer = np.concatenate((lgbm1_train, lgbm2_train, lgbm3_train,
                         lgbm4_train, xgb1_train, xgb2_train,
                         gbm1_train,ridge_train, lasso_train, rf_train,et_train), axis=1)
x_test_second_layer = np.concatenate((lgbm1_test, lgbm2_test, lgbm3_test,
                        lgbm4_test,xgb1_test,xgb2_test,
                       gbm1_test,ridge_test,lasso_test,rf_test,et_test), axis=1)

In [ ]:
lgbm_ex_no = 9
lgbm_meta_params = {
    'boosting':'gbdt', 'num_leaves':28, 'learning_rate':0.03, 'min_sum_hessian_in_leaf':0.1,
    'max_depth':7, 'feature_fraction':0.6, 'min_data_in_leaf':70, 'poission_max_delta_step':0.7,
    'bagging_fraction':0.8, 'min_gain_to_split':0, 'scale_pos_weight':1.0,
    'lambda_l2':0.1, 'lambda_l1':0.1, 'fair_c':1.0, 'bagging_freq':1,
    'objective':'fair', 'seed':1, 'categorical_feature':0, 'xgboost_dart_mode':False,
    'drop_rate':0.1, 'skip_drop':0.5, 'max_drop':50, 'top_rate':0.1, 'other_rate':0.1,
    'max_bin':255, 'min_data_in_bin':50, 'bin_construct_sample_cnt':1000000,
    'two_round':False, 'uniform_drop':False,'metric': 'mae','threads':6
}

lgbm_meta_model = stacking.LgbmWrapper(params=lgbm_meta_params, num_rounds = 2000, ealry_stopping=100,
               verbose_eval=False, base_score=True, maximize=False, y_value_log=False)

lgbm_cv_score,best_round = stacking.kfold_test(lgbm_meta_model, pd.DataFrame(x_train_second_layer), 
                                               y_train,  mean_absolute_error,NFOLDS=5 )


d_train_all = lgbm.Dataset(pd.DataFrame(x_train_second_layer), label=y_train)
bst = lgbm.train(lgbm_params,d_train_all,best_round)
predictions = bst.predict(pd.DataFrame(x_test_second_layer))

In [ ]:
print("fscore result")
fscore_df = pd.concat([pd.DataFrame(bst.feature_name()),pd.DataFrame(bst.feature_importance())],axis=1)
fscore_df.columns = ['column','fscore']
fscore_df['fscore'] = fscore_df['fscore'].astype(int)
fscore_df.sort_values(by='fscore',ascending=False,inplace=True)
fscoe_output = 'fscore\\ex_'+str(lgbm_ex_no)+'_lgbm_fscore_'+ str(lgbm_cv_score)+ '.csv'
fscore_df.to_csv(fscoe_output)

fig, ax = plt.subplots(figsize=(12,18))
lgbm.plot_importance(bst, height=0.8, ax=ax)
plt.show()

In [ ]:
print("Predict")
sub = pd.read_csv('input/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    print(c)
    sub[c] = predictions

print("Wrting Files")
sub_output = 'output\\ex_'+str(lgbm_ex_no)+'_lightgbm_'+str(lgbm_cv_score)
sub.to_csv(sub_output+'.csv', index=False, float_format='%.4f') # Thanks to @inversion

print("File Zip")
jungle_zip = zipfile.ZipFile(sub_output +'.zip', 'w')
jungle_zip.write(sub_output + '.csv', compress_type=zipfile.ZIP_DEFLATED)
 
jungle_zip.close()
print("Done")